In [0]:
%cd /content/drive/My Drive/Insight/Project/fashionbeans/processedimages/processed

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install keras-maskrcnn

In [0]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras
import keras_maskrcnn
import keras_retinanet 

# import keras_retinanet
from keras_maskrcnn import models
from keras_maskrcnn.utils.visualization import draw_mask
from keras_retinanet.utils.visualization import draw_box, draw_caption, draw_annotations
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import shutil 
import numpy as np
import time
import json



# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
# adjust this to point to your downloaded/trained model
model_path = os.path.join('/content/drive/My Drive/Insight/Project/MAsk-RCNN', 'resnet50_modanet.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {1: 'bag', 2: 'belt', 3: 'boots', 4: 'footwear', 5: 'outer', 6: 'dress', 7: 'sunglasses', 8: 'pants', 9: 'top', 10: 'shorts', 11: 'skirt', 12: 'headwear', 13: 'scarf/tie'}

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:335: UserWarning: Output "filtered_detections" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "filtered_detections" during training.
  sample_weight_mode=sample_weight_mode)
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:335: UserWarning: Output "mask_submodel" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "mask_submodel" during training.
  sample_weight_mode=sample_weight_mode)
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWar

In [0]:
%pwd

'/content/drive/My Drive/Insight/Project/fashionbeans/processedimages/processed'

In [0]:
def cloth_identifier(input_imagefile):
  # load image
  image = read_image_bgr(input_imagefile)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # preprocess image for network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # process image
  start = time.time()
  outputs = model.predict_on_batch(np.expand_dims(image, axis=0))
  print("processing time: ", time.time() - start)

  boxes  = outputs[-4][0]
  scores = outputs[-3][0]
  labels = outputs[-2][0]
  masks  = outputs[-1][0]

  # correct for image scale
  boxes /= scale

  masks_dic={}
  boxes_dic={}
  counter=0

  # visualize detections
  for box, score, label, mask in zip(boxes, scores, labels, masks):
    if score < 0.5:
        break
    

    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    mask = mask[:, :, label]
    draw_mask(draw, b, mask, color=label_color(label))
    
    masks_dic[str(counter)]=mask
    boxes_dic[str(counter)]=box

    counter+=1

    #caption = "{} {:.3f}".format(labels_to_names[label], score)
    #draw_caption(draw, b, caption)
    
  #newfileneame=input_imagefile.split(".")[0]  
  #plt.ioff()
  #plt.figure(figsize=(15, 15))
  #plt.axis('off')
  #plt.imshow(draw)
  #plt.savefig(str('maskedimages/')+str(newfileneame)+'-masked'+'.jpg',bbox_inches='tight', pad_inches=0)
  #plt.close('all')
  #cwd = os.getcwd()

  
  #print(str(cwd)+'/'+'masked'+str(newfileneame)+'.jpg')
  #plt.show()
  

  

  return draw, masks_dic, boxes_dic



In [0]:
new_img, masks_dic, boxes_dic = cloth_identifier('42+23.jpg')

processing time:  4.003019332885742


In [0]:
### This is the function to extract clothing items separately from an image, given 
### the image and masks and boxes (as dictionaries) extracted using the function 'cloth_identifier'
### and returns a separate image file for each identfied clothing, based on the masks

def cloth_extract(input_imagefile, masks_dic, boxes_dic):
  # load image
  image = read_image_bgr(input_imagefile)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # visualize detections

  items_dic={}
  counter=0

  for box, mask in zip(boxes_dic.values(), masks_dic.values()):

    b = box.astype(int)

    # resize to fit the box
    mask = mask.astype(np.float32)
    mask = cv2.resize(mask, (b[2] - b[0], b[3] - b[1]))

    # binarize the mask
    mask = (mask > 0.5).astype(np.uint8)

    # draw the mask in the image
    mask_image = np.zeros((draw.shape[0], draw.shape[1]), np.uint8)
    mask_image[b[1]:b[3], b[0]:b[2]] = mask
    mask = mask_image

    mask = (np.stack([mask] * 3, axis = 2))*draw

    items_dic[str(counter)] = mask
    counter+=1

    newfileneame=input_imagefile.split(".")[0]
    plt.ioff()
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(mask)
    plt.savefig(str('/content/drive/My Drive/Insight/Project/fashionbeans/maskeditems/')+str(newfileneame)+'-masked-'+str(counter)+'.jpg',bbox_inches='tight', pad_inches=0)
    #plt.show()
    plt.close('all')
    
  
  return items_dic




In [0]:
items_dic = cloth_extract('0+1.jpg', masks_dic, boxes_dic)

In [0]:
### To run cloth_identifier and cloth_extract over all images in a directory 

directory = '/content/drive/My Drive/Insight/Project/fashionbeans/processedimages/processed'
counter=0
onlyfiles = next(os.walk(directory))[2]
masked_images_dic={}
masked_items_dic={}

for filename in os.listdir(directory):
  if filename.endswith('jpg') or filename.endswith('png'):
    print(filename)

    #identifying the clothing
    new_img, masks_dic, boxes_dic = cloth_identifier(filename)
    #extracting the items
    items_dic = cloth_extract(filename, masks_dic, boxes_dic)
                                        
                          
    #masked_images_dic[key]=(new_img)
    #masked_items_dic[key]=(items_dic)
                                  
    counter+=1
                                        
    #moving porcessed image into /processedimages/
    cwd = os.getcwd()                                    
    os.rename('/content/drive/My Drive/Insight/Project/fashionbeans/processedimages/processed/'+str(filename), '/content/drive/My Drive/Insight/Project/fashionbeans/processedimages/'+str(filename))

    print("File#",counter,"of",len(onlyfiles),"is processed")


42+23.jpg
processing time:  0.7416341304779053
File# 1 of 2280 is processed
42+24.jpg
processing time:  0.7533218860626221
File# 2 of 2280 is processed
42+25.jpg
processing time:  0.7530064582824707
File# 3 of 2280 is processed
42+26.jpg
processing time:  0.7414119243621826
File# 4 of 2280 is processed
42+27.jpg
processing time:  0.7317211627960205
File# 5 of 2280 is processed
42+28.jpg
processing time:  0.7237749099731445
File# 6 of 2280 is processed
42+29.jpg
processing time:  0.7212815284729004
File# 7 of 2280 is processed
42+30.jpg
processing time:  0.7291934490203857
File# 8 of 2280 is processed
43+1.jpg
processing time:  0.7441990375518799
File# 9 of 2280 is processed
43+2.jpg
processing time:  0.7229604721069336
File# 10 of 2280 is processed
43+3.jpg
processing time:  0.7231385707855225
File# 11 of 2280 is processed
43+4.jpg
processing time:  0.7215602397918701
File# 12 of 2280 is processed
43+5.jpg
processing time:  0.7397050857543945
File# 13 of 2280 is processed
43+6.jpg
proc

In [0]:
import os
import glob

files = glob.glob('/content/drive/My Drive/Insight/Project/fashionbeans/')
for f in files:
  if f.endswith('jpg'):
    print(f)
    os.remove(f)